In [76]:
import glob 
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import numpy as np
from datetime import datetime
import sqlite3

In [77]:
url='https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
table_Attributes=['Name','MC_USD_Billion','MC_GBP_Billion','MC_EUR_Billion','MC_INR_Billion']
df = pd.DataFrame(columns=table_Attributes)
table_name = 'Largest_banks'
csv_path = './BAnks.csv'


In [78]:
def extract(url, table_Attributes):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_Attributes)
    tables = data.find_all('table')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Name": col[1].get_text(strip=True),
                             "MC_USD_Billion": col[2].get_text(strip=True)}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
    return df


In [91]:
extract(url,table_Attributes)


,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,NaN,NaN,NaN
1,Bank of America,231.52,NaN,NaN,NaN
2,Industrial and Commercial Bank of China,194.56,NaN,NaN,NaN
3,Agricultural Bank of China,160.68,NaN,NaN,NaN
4,HDFC Bank,157.91,NaN,NaN,NaN
5,Wells Fargo,155.87,NaN,NaN,NaN
6,HSBC Holdings PLC,148.90,NaN,NaN,NaN
7,Morgan Stanley,140.83,NaN,NaN,NaN
8,China Construction Bank,139.82,NaN,NaN,NaN
9,Bank of China,136.81,NaN,NaN,NaN


In [80]:
def transform(df):
    MC_USD_list = df["MC_USD_Billion"].tolist()
    MC_USD_list= [float(x) for x in MC_USD_list]
    MC_USD_list1 = [np.round(x*0.93,2) for x in MC_USD_list]
    df["MC_EUR_Billion"] = MC_USD_list1
    MC_USD_list2= [np.round(x*0.8295,2) for x in MC_USD_list]
    df["MC_INR_Billion"] = MC_USD_list2
    MC_USD_list3= [np.round(x*0.8,2) for x in MC_USD_list]
    df["MC_GBP_Billion"] = MC_USD_list3
    return df


In [81]:
transform(df)

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.34,402.62,359.11
1,Bank of America,231.52,185.22,215.31,192.05
2,Industrial and Commercial Bank of China,194.56,155.65,180.94,161.39
3,Agricultural Bank of China,160.68,128.54,149.43,133.28
4,HDFC Bank,157.91,126.33,146.86,130.99
5,Wells Fargo,155.87,124.70,144.96,129.29
6,HSBC Holdings PLC,148.90,119.12,138.48,123.51
7,Morgan Stanley,140.83,112.66,130.97,116.82
8,China Construction Bank,139.82,111.86,130.03,115.98
9,Bank of China,136.81,109.45,127.23,113.48


In [92]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)
def load_to_db(df,sql_connection,table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [83]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [84]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./etl_project_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [90]:
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, table_Attributes)
log_progress('Data extraction complete. Initiating Transformation process')
df =transform(df)
log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('World_Economies.db')
log_progress('SQL Connection initiated.')
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement, sql_connection)
query_statement1 = f"SELECT name FROM {table_name}"
run_query(query_statement1, sql_connection)
query_statement2 = f"SELECT * FROM {table_name}"
run_query(query_statement2, sql_connection)
log_progress('Process Complete.')
sql_connection.close()



SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987
SELECT name FROM Largest_banks
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank
5                              Wells Fargo
6                        HSBC Holdings PLC
7                           Morgan Stanley
8                  China Construction Bank
9                            Bank of China
SELECT * FROM Largest_banks
                                      Name MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase         432.92          346.34   
1                          Bank of America         231.52          185.22   
2  Industrial and Commercial Bank of China         194.56          155.65   
3               Agricultural Bank of China         160.68          1